In [66]:
import numpy as np
import math
 

class Tensor():

    def __init__(self, data, _children = ()):

        self.data = np.array(data, dtype =np.float32)
        self.shape = self.data.shape
        self._backward = lambda : None
        self.prev = set(_children)
        self.grad = 0.0

    def __repr__(self):
        return f"<Tensor data = {self.data}>"
    
    def shape(self): return self.shape
    # -----------------------------------------  binary OPS ----------------------------------------------------
    def __add__(self, other):   #addition of Tensor by a int(float) or a Tensror of the same size(element wise)
        
        other = other if isinstance(other, Tensor) else Tensor(other)
        output_T  = Tensor(self.data + other.data, (self,other))

        def _backward():
            self.grad += output_T.grad  * 1.0  # chain rule 
            other.grad += output_T.grad * 1.0

        output_T._backward = _backward
        return output_T

    def __mul__(self, other): #multiplication of Tensor by a int(float) or a Tensror of the same size(element wise)

        other = other if isinstance(other, Tensor) else Tensor(other)
        output_T = Tensor(self.data * other.data,(self, other))

        def _backward():
            self.grad += (other.data * output_T.grad) 
            other.grad += (self.data * output_T.grad)
    
        output_T._backward = _backward
        return output_T

    def suma(self,axis = None):
        output_T = Tensor(self.data.sum(axis = axis))

        def _backward():
            self.grad = self.data * output_T.grad

        output_T._backward = _backward
        return output_T 
        
    def backward(self):
    
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v.prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1

        for node in reversed(topo):
            node._backward()

In [75]:
t = Tensor([[0.6],[0.2]])
k = Tensor([[1.2],[0.3]])
o = Tensor([[2.0],[3.0]])
print(t.shape)
first  = t+ k
second = o * first
L = second.suma()
print(L)


(2, 1)
<Tensor data = [[3.6000001]
 [1.5      ]]>


<Tensor data = [[3.6000001]
 [1.5      ]]>